In [1]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.2. Original Data')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.1. PDF Data Extraction/Output')
cur_date = "011422"

library(tidyverse)
library(pdftools)
library(tm)

#only grabs files ending in "pdf"
pdf_list = list.files(pattern = "pdf$")
head(pdf_list)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Using poppler version 20.12.1

Loading required package: NLP


Attaching package: ‘NLP’


The following object is masked from ‘package:ggplot2’:

    annotate




[1] "20210910_0000_9.3.1._4_size.pdf"  "20210910_0001_9.3.1._1_size.pdf" 
[3] "20210910_0002_9.3.1._7_size.pdf"  "20210910_0003_9.3.1._10_size.pdf"
[5] "20210914_0049_9.3.1_2_size.pdf"   "20210914_0050_9.3.1_5_size.pdf"

# Data we want to extract:

1. Pdf file name as the sample identifier
2. Original Concentration
3. Some of the ‘X Values’ table listed on the bottom left, specifically the: X10 Concentration, X50 Concentration, X90 Concentration, and Mean Concentration, and StdDev Concentration

In [2]:
corpus_raw <- data.frame("Sample Identifier" = c(),"Text" = c())

for (i in 1:length(pdf_list)){
    document_text = pdf_text(paste(pdf_list[i],sep = "")) %>% 
    strsplit("\n")
    
        document = data.frame("Sample Identifier" = gsub(x =pdf_list[i],pattern = ".pdf", replacement = ""), 
        "Text" = document_text, stringsAsFactors = FALSE)
    
    colnames(document) <- c("Sample Identifier", "Text")
    corpus_raw <- rbind(corpus_raw,document) 
}

#still haven't figured out why there are duplicates
corpus_raw = unique(corpus_raw)

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: Invalid least number of objects reading page offset hints table

PDF error: I

In [3]:
head(corpus_raw)

,Sample Identifier,Text
,<chr>,<chr>
1,20210910_0000_9.3.1._4_size,Electrophoresis & Brownian Motion
2,20210910_0000_9.3.1._4_size,Video Analysis
3,20210910_0000_9.3.1._4_size,Laser Scattering Microscopy
4,20210910_0000_9.3.1._4_size,www.particle-metrix.de
5,20210910_0000_9.3.1._4_size,
6,20210910_0000_9.3.1._4_size,Operator (Report): sop_kabanovlab.inst


In [4]:
OG_concentration_df = corpus_raw %>%
    filter(grepl("Original Concentration", Text)) %>%
    separate(Text, c(NA, "Original Concentration"), sep = ": ") %>%
    separate(`Original Concentration`, c("Original Concentration (Particles/ mL)", NA), sep = "P")
head(OG_concentration_df)

,Sample Identifier,Original Concentration (Particles/ mL)
,<chr>,<chr>
1,20210910_0000_9.3.1._4_size,4.9E+9
2,20210910_0001_9.3.1._1_size,3.1E+9
3,20210910_0002_9.3.1._7_size,2.2E+9
4,20210910_0003_9.3.1._10_size,1.3E+9
5,20210914_0049_9.3.1_2_size,1.8E+9
6,20210914_0050_9.3.1_5_size,NaN


In [5]:
X_concentration_df = corpus_raw %>%
    filter(grepl("X10", Text)| grepl("X50 ", Text)| grepl("X90", Text) | grepl("Mean", Text)| grepl("StdDev", Text)) %>%
    separate(Text, c("Text", NA), sep = "                ") %>%
    separate(Text, c("Text", "Concentration"), sep = "      ") %>%
    pivot_wider(names_from = Text, values_from = Concentration)
head(X_concentration_df)

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 4 rows [26, 27, 28, 29].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 4 rows [26, 27, 28, 29].”


Sample Identifier,X10,X50,X90,Mean,StdDev
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
20210910_0000_9.3.1._4_size,57.3,127.2,270.8,162.6,90.1
20210910_0001_9.3.1._1_size,63.2,121.1,247.7,159.3,84.6
20210910_0002_9.3.1._7_size,72.9,150.0,297.0,181.3,84.3
20210910_0003_9.3.1._10_size,59.2,152.0,304.5,190.9,109.2
20210914_0049_9.3.1_2_size,72.0,115.8,216.6,135.3,59.3
20210914_0050_9.3.1_5_size,NA,NA,NA,NA,NA


In [6]:
#combining into one final df
final_df = full_join(OG_concentration_df, X_concentration_df)
head(final_df)

Joining, by = "Sample Identifier"



,Sample Identifier,Original Concentration (Particles/ mL),X10,X50,X90,Mean,StdDev
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,20210910_0000_9.3.1._4_size,4.9E+9,57.3,127.2,270.8,162.6,90.1
2,20210910_0001_9.3.1._1_size,3.1E+9,63.2,121.1,247.7,159.3,84.6
3,20210910_0002_9.3.1._7_size,2.2E+9,72.9,150.0,297.0,181.3,84.3
4,20210910_0003_9.3.1._10_size,1.3E+9,59.2,152.0,304.5,190.9,109.2
5,20210914_0049_9.3.1_2_size,1.8E+9,72.0,115.8,216.6,135.3,59.3
6,20210914_0050_9.3.1_5_size,NaN,NA,NA,NA,NA,NA


In [7]:
#exporting data
write.csv(final_df, paste0(Output,"/", cur_date, "_Final_PDF_Data.csv"), row.names = FALSE)